# Imports

In [1]:
import pandas as pd 
import numpy as np 

# Constants

In [2]:
TRAIN_DATA_FILE = 'SpamData/SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/SpamData/03_Testing/prob-ham.txt'
TOKEN_ALLPROB_FILE = 'SpamData/SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURES_MATRIX = 'SpamData/SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

## txt to numpy

In [3]:
sparse_train_data = np.loadtxt(TRAIN_DATA_FILE, delimiter=' ', dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
def make_full_matrix(sparse_matrix, nr_words=VOCAB_SIZE, doc_idx=0, word_idx=1, category_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Returns a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- vocabulary size
    doc_idx -- position in the sparse matrix
    word_idx -- position in the sparse matrix
    category_idx -- position in the sparse matrix
    freq_idx -- position in the sparse matrix
    
    """
    
    columns_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=columns_names)
    full_matrix.fillna(value=0, inplace=True)
    
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        category = sparse_matrix[i][category_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = category
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
        
        
    return full_matrix

In [5]:
%%time

full_train_data = make_full_matrix(sparse_train_data, nr_words=VOCAB_SIZE)

CPU times: total: 23.8 s
Wall time: 38.9 s


### spam prob

In [6]:
prob_sum = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
prob_sum

0.310989284824321

### total number of words

In [7]:
full_train_features = full_train_data.loc[:, full_train_data.columns!='CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
email_length = full_train_features.sum(axis=1)
total_words = email_length.sum()

In [9]:
spam_lengths = email_length[full_train_data.CATEGORY == 1]
ham_lengths = email_length[full_train_data.CATEGORY == 0]

spam_total_words = spam_lengths.sum()
ham_total_words = ham_lengths.sum()


In [10]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 

train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1 


# probabilities #

## P( Token | Spam)

In [11]:
prob_tokens_spam = summed_spam_tokens / (spam_total_words + VOCAB_SIZE)

## P( Token | Spam)

In [12]:
prob_tokens_ham = summed_ham_tokens / (ham_total_words + VOCAB_SIZE)

## P(Token)

In [13]:
probs_token_all = full_train_features.sum(axis=0) / total_words

In [14]:
np.savetxt(TOKEN_ALLPROB_FILE, probs_token_all)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)

In [15]:
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

In [16]:
x_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [17]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURES_MATRIX, x_test)